## 缺失值的处理


In [1]:
import numpy as np
import pandas as pd


In [5]:
data=pd.read_csv("C:/Users/gen'ch/pandas学习/joyful-pandas-master/data/learn_pandas.csv",
                usecols = ['Grade', 'Name', 'Gender', 'Height','Weight', 'Transfer'])

In [6]:
data.head()

,Grade,Name,Gender,Height,Weight,Transfer
0,Freshman,Gaopeng Yang,Female,158.9,46.0,N
1,Freshman,Changqiang You,Male,166.5,70.0,N
2,Senior,Mei Sun,Male,188.9,89.0,N
3,Sophomore,Xiaojuan Sun,Female,NaN,41.0,N
4,Sophomore,Gaojuan You,Male,174.0,74.0,N


#### 查询缺失值

In [7]:
data.isna().head()

,Grade,Name,Gender,Height,Weight,Transfer
0,False,False,False,False,False,False
1,False,False,False,False,False,False
2,False,False,False,False,False,False
3,False,False,False,True,False,False
4,False,False,False,False,False,False


In [8]:
data.isna().mean()

Grade       0.000
Name        0.000
Gender      0.000
Height      0.085
Weight      0.055
Transfer    0.060
dtype: float64

In [9]:
data[data.Height.isna()].head()

,Grade,Name,Gender,Height,Weight,Transfer
3,Sophomore,Xiaojuan Sun,Female,NaN,41.0,N
12,Senior,Peng You,Female,NaN,48.0,NaN
26,Junior,Yanli You,Female,NaN,48.0,N
36,Freshman,Xiaojuan Qin,Male,NaN,79.0,Y
60,Freshman,Yanpeng Lv,Male,NaN,65.0,N


In [11]:
sub_set = data[['Height', 'Weight', 'Transfer']]

In [12]:
data[sub_set.isna().all(1)]

,Grade,Name,Gender,Height,Weight,Transfer
102,Junior,Chengli Zhao,Male,NaN,NaN,NaN


In [13]:
data[sub_set.isna().any(1)].head()

,Grade,Name,Gender,Height,Weight,Transfer
3,Sophomore,Xiaojuan Sun,Female,NaN,41.0,N
9,Junior,Juan Xu,Female,164.8,NaN,N
12,Senior,Peng You,Female,NaN,48.0,NaN
21,Senior,Xiaopeng Shen,Male,166.0,62.0,NaN
26,Junior,Yanli You,Female,NaN,48.0,N


In [14]:
data[sub_set.notna().all(1)].head()

,Grade,Name,Gender,Height,Weight,Transfer
0,Freshman,Gaopeng Yang,Female,158.9,46.0,N
1,Freshman,Changqiang You,Male,166.5,70.0,N
2,Senior,Mei Sun,Male,188.9,89.0,N
4,Sophomore,Gaojuan You,Male,174.0,74.0,N
5,Freshman,Xiaoli Qian,Female,158.0,51.0,N


### 缺失信息和删除
dropna 的主要参数为轴方向 axis （默认为0，即删除行）、删除方式 how 、删除的非缺失值个数阈值 thresh （ 非缺失值 没有达到这个数量的相应维度会被删除）、备选的删除子集 subset ，其中 how 主要有 any 和 all 两种参数可以选择。

In [15]:
res = data.dropna(how = 'any', subset = ['Height', 'Weight'])

In [17]:
res.head()

,Grade,Name,Gender,Height,Weight,Transfer
0,Freshman,Gaopeng Yang,Female,158.9,46.0,N
1,Freshman,Changqiang You,Male,166.5,70.0,N
2,Senior,Mei Sun,Male,188.9,89.0,N
4,Sophomore,Gaojuan You,Male,174.0,74.0,N
5,Freshman,Xiaoli Qian,Female,158.0,51.0,N


In [18]:
res = data.dropna(1, thresh=data.shape[0]-15)

In [19]:
res = data.loc[data[['Height', 'Weight']].notna().all(1)]

In [20]:
res.shape

(174, 6)

In [21]:
res = data.loc[:, ~(data.isna().sum()>15)]

In [22]:
res.head()

,Grade,Name,Gender,Weight,Transfer
0,Freshman,Gaopeng Yang,Female,46.0,N
1,Freshman,Changqiang You,Male,70.0,N
2,Senior,Mei Sun,Male,89.0,N
3,Sophomore,Xiaojuan Sun,Female,41.0,N
4,Sophomore,Gaojuan You,Male,74.0,N


### 缺失值的填充和插值
- 利用fillna进行填充

在 fillna 中有三个参数是常用的： value, method, limit 。其中， value 为填充值，可以是标量，也可以是索引到元素的字典映射； method 为填充方法，有用前面的元素填充 ffill 和用后面的元素填充 bfill 两种类型， limit 参数表示连续缺失值的最大填充次数

In [23]:
s = pd.Series([np.nan, 1, np.nan, np.nan, 2, np.nan],list('aaabcd'))

In [24]:
s

a    NaN
a    1.0
a    NaN
b    NaN
c    2.0
d    NaN
dtype: float64

In [25]:
s.fillna(method='ffill')

a    NaN
a    1.0
a    1.0
b    1.0
c    2.0
d    2.0
dtype: float64

In [26]:
s.fillna(method='ffill', limit=1)

a    NaN
a    1.0
a    1.0
b    NaN
c    2.0
d    2.0
dtype: float64

In [27]:
s.fillna(s.mean())

a    1.5
a    1.0
a    1.5
b    1.5
c    2.0
d    1.5
dtype: float64

In [28]:
s.fillna({'a': 100, 'd': 200})

a    100.0
a      1.0
a    100.0
b      NaN
c      2.0
d    200.0
dtype: float64

In [29]:
data.groupby('Grade')['Height'].transform(lambda x: x.fillna(x.mean())).head()

0    158.900000
1    166.500000
2    188.900000
3    163.075862
4    174.000000
Name: Height, dtype: float64

### 插值函数
在关于 interpolate 函数的 文档 描述中，列举了许多插值法，包括了大量 Scipy 中的方法。由于很多插值方法涉及到比较复杂的数学知识，因此这里只讨论比较常用且简单的三类情况，即线性插值、最近邻插值和索引插值。

对于 interpolate 而言，除了插值方法（默认为 linear 线性插值）之外，有与 fillna 类似的两个常用参数，一个是控制方向的 limit_direction ，另一个是控制最大连续缺失值插值个数的 limit 。其中，限制插值的方向默认为 forward ，这与 fillna 的 method 中的 ffill 是类似的，若想要后向限制插值或者双向限制插值可以指定为 backward 或 both 。

In [30]:
s = pd.Series([np.nan, np.nan, 1,np.nan, np.nan, np.nan,2, np.nan, np.nan])

In [32]:
s

0    NaN
1    NaN
2    1.0
3    NaN
4    NaN
5    NaN
6    2.0
7    NaN
8    NaN
dtype: float64

In [31]:
res = s.interpolate(limit_direction='backward', limit=1)

In [33]:
res.values

array([ nan, 1.  , 1.  ,  nan,  nan, 1.75, 2.  ,  nan,  nan])

In [34]:
res = s.interpolate(limit_direction='both', limit=1)

In [35]:
res.values

array([ nan, 1.  , 1.  , 1.25,  nan, 1.75, 2.  , 2.  ,  nan])

In [36]:
s.interpolate('nearest').values

array([nan, nan,  1.,  1.,  1.,  2.,  2., nan, nan])

In [37]:
s = pd.Series([0,np.nan,10],index=[0,1,10])

In [38]:
s

0      0.0
1      NaN
10    10.0
dtype: float64

In [39]:
s.interpolate()

0      0.0
1      5.0
10    10.0
dtype: float64

In [40]:
s.interpolate(method='index')

0      0.0
1      1.0
10    10.0
dtype: float64

In [41]:
s = pd.Series([0,np.nan,10],index=pd.to_datetime(['20200101',
                                                  '20200102',
                                                  '20200111']))

In [42]:
s

2020-01-01     0.0
2020-01-02     NaN
2020-01-11    10.0
dtype: float64

In [43]:
s.interpolate()

2020-01-01     0.0
2020-01-02     5.0
2020-01-11    10.0
dtype: float64

In [44]:
s.interpolate(method='index')

2020-01-01     0.0
2020-01-02     1.0
2020-01-11    10.0
dtype: float64

## nullable类型
- 1、缺失记号及其缺陷

In [45]:
None == None

True

In [46]:
None == False

False

In [47]:
None == []

False

In [48]:
None == ''

False

在 numpy 中利用 np.nan 来表示缺失值，该元素除了不和其他任何元素相等之外，和自身的比较结果也返回 False 

In [49]:
np.nan == np.nan

False

In [50]:
np.nan == None

False

In [51]:
np.nan == False

False

值得注意的是，虽然在对缺失序列或表格的元素进行比较操作的时候， np.nan 的对应位置会返回 False ，但是在使用 equals 函数进行两张表或两个序列的相同性检验时，会自动跳过两侧表都是缺失值的位置，直接返回 True

In [52]:
s1 = pd.Series([1, np.nan])

In [53]:
s2 = pd.Series([1, 2])

In [54]:
s3 = pd.Series([1, np.nan])

In [55]:
s1 == 1

0     True
1    False
dtype: bool

In [56]:
s1.equals(s2)

False

In [57]:
s1.equals(s3)

True

In [58]:
pd.to_timedelta(['30s', np.nan])

TimedeltaIndex(['0 days 00:00:30', NaT], dtype='timedelta64[ns]', freq=None)

In [59]:
pd.to_datetime(['20200101', np.nan])

DatetimeIndex(['2020-01-01', 'NaT'], dtype='datetime64[ns]', freq=None)

在 pandas 中可以看到 object 类型的对象，而 object 是一种混杂对象类型，如果出现了多个类型的元素同时存储在 Series 中，它的类型就会变成 object 

In [60]:
pd.Series([1, 'two'])

0      1
1    two
dtype: object

NaT 问题的根源来自于 np.nan 的本身是一种浮点类型，而如果浮点和时间类型混合存储，如果不设计新的内置缺失类型来处理，就会变成含糊不清的 object 类型，这显然是不希望看到的

In [61]:
type(np.nan)

float

同时，由于 np.nan 的浮点性质，如果在一个整数的 Series 中出现缺失，那么其类型会转变为 float64 ；而如果在一个布尔类型的序列中出现缺失，那么其类型就会转为 object 而不是 bool 

In [62]:
pd.Series([1, np.nan]).dtype

dtype('float64')

In [63]:
pd.Series([True, False, np.nan]).dtype

dtype('O')

### Nullable类型的性质
从字面意义上看 Nullable 就是可空的，言下之意就是序列类型不受缺失值的影响。例如，在上述三个 Nullable 类型中存储缺失值，都会转为 pandas 内置的 pd.NA 

In [64]:
pd.Series([np.nan, 1], dtype = 'Int64')

0    <NA>
1       1
dtype: Int64

In [65]:
pd.Series([np.nan, True], dtype = 'boolean')

0    <NA>
1    True
dtype: boolean

In [66]:
pd.Series([np.nan, 'my_str'], dtype = 'string')

0      <NA>
1    my_str
dtype: string

In [67]:
pd.Series([np.nan, 0], dtype = 'Int64') + 1

0    <NA>
1       1
dtype: Int64

In [68]:
pd.Series([np.nan, 0], dtype = 'Int64') == 0

0    <NA>
1    True
dtype: boolean

In [69]:
pd.Series([np.nan, 0], dtype = 'Int64') * 0.5

0    NaN
1    0.0
dtype: float64

对于 boolean 类型的序列而言，其和 bool 序列的行为主要有两点区别：

第一点是带有缺失的布尔列表无法进行索引器中的选择，而 boolean 会把缺失值看作 False ：

In [70]:
s = pd.Series(['a', 'b'])

In [71]:
s_bool = pd.Series([True, np.nan])

In [72]:
s_boolean = pd.Series([True, np.nan]).astype('boolean')

In [73]:
s[s_boolean]

0    a
dtype: object

第二点是在进行逻辑运算时， bool 类型在缺失处返回的永远是 False ，而 boolean 会根据逻辑运算是否能确定唯一结果来返回相应的值。

In [74]:
s_boolean & True

0    True
1    <NA>
dtype: boolean

In [75]:
s_boolean | True

0    True
1    True
dtype: boolean

In [76]:
~s_boolean

0    False
1     <NA>
dtype: boolean

一般在实际数据处理时，可以在数据集读入后，先通过 convert_dtypes 转为 Nullable 类型：

In [77]:
data1 = pd.read_csv("C:/Users/gen'ch/pandas学习/joyful-pandas-master/data/learn_pandas.csv")


In [78]:
data1=data1.convert_dtypes()

In [79]:
data1.dtypes

School          string
Grade           string
Name            string
Gender          string
Height         float64
Weight           Int64
Transfer        string
Test_Number      Int64
Test_Date       string
Time_Record     string
dtype: object

### 缺失数据的计算和分组
当调用函数 sum, prob 使用加法和乘法的时候，缺失数据等价于被分别视作0和1，即不改变原来的计算结果

In [80]:
s = pd.Series([2,3,np.nan,4,5])

In [81]:
s.sum()

14.0

In [82]:
s.prod()

120.0

In [83]:
s.cumsum()

0     2.0
1     5.0
2     NaN
3     9.0
4    14.0
dtype: float64

当进行单个标量运算的时候，除了 np.nan ** 0 和 1 ** np.nan 这两种情况为确定的值之外，所有运算结果全为缺失（ pd.NA 的行为与此一致 ），并且 np.nan 在比较操作时一定返回 False ，而 pd.NA 返回 pd.NA

In [84]:
np.nan == 0

False

In [85]:
pd.NA == 0

<NA>

In [86]:
np.nan > 0

False

In [87]:
pd.NA > 0

<NA>

In [88]:
np.nan + 1

nan

In [89]:
np.log(np.nan)

nan

In [90]:
np.add(np.nan, 1)

nan

In [91]:
np.nan ** 0

1.0

In [92]:
pd.NA ** 0

1

In [93]:
1 ** np.nan

1.0

In [94]:
1 ** pd.NA

1

 diff, pct_change 这两个函数虽然功能相似，但是对于缺失的处理不同，前者凡是参与缺失计算的部分全部设为了缺失值，而后者缺失值位置会被设为 0% 的变化率

In [95]:
s.diff()

0    NaN
1    1.0
2    NaN
3    NaN
4    1.0
dtype: float64

In [96]:
s.pct_change()

0         NaN
1    0.500000
2    0.000000
3    0.333333
4    0.250000
dtype: float64

In [97]:
df_nan = pd.DataFrame({'category':['a','a','b',np.nan,np.nan],'value':[1,3,5,7,9]})

In [98]:
df_nan

,category,value
0,a,1
1,a,3
2,b,5
3,NaN,7
4,NaN,9


In [99]:
df_nan.groupby('category',dropna=False)['value'].mean()

category
a      2
b      5
NaN    8
Name: value, dtype: int64

In [100]:
pd.get_dummies(df_nan.category, dummy_na=True)

,a,b,NaN
0,1,0,0
1,1,0,0
2,0,1,0
3,0,0,1
4,0,0,1
